## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
#tar zcvf dynamic-ab-testing.tar.gz dynamic-ab-testing/
#!aws s3 cp dynamic-ab-testing.gz s3://dat-bucket-dongjin
#aws s3 cp s3://dat-bucket-dongjin/dynamic-ab-testing.tar.gz .
#ar zxvf dynamic-ab-testing.tar.gz 

## 0. Install requirements

In [2]:
%%sh
bash ./dependency/setup.sh

~/SageMaker/dynamic-ab-testing/dependency ~/SageMaker/dynamic-ab-testing

=== Environment Settting for Dynamic AB-Testing with MLOps ===
    WORKDIR: /home/ec2-user/SageMaker/dynamic-ab-testing/dependency 

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com

=== Done ===
~/SageMaker/dynamic-ab-testing


## 1. Set roles

In [3]:
import boto3
import sagemaker
from utils.iam import iam_handler
from utils.ssm import parameter_store
from sagemaker import get_execution_role

In [4]:
strRegionName=boto3.Session().region_name
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]

In [5]:
iam = iam_handler()

In [ ]:
#iam role은 어떻게 붙일 수 있나?
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess",
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

In [ ]:
pm = parameter_store(strRegionName)

In [6]:
pm.put_params(key="DAT-REGION", value=strRegionName, overwrite=True)
pm.put_params(key="DAT-ACCOUNT-ID", value=boto3.client("sts").get_caller_identity().get("Account"), overwrite=True)
pm.put_params(key="DAT-BUCKET", value=sagemaker.Session().default_bucket(), overwrite=True)
pm.put_params(key="DAT-SAGEMAKER-ROLE-ARN", value=get_execution_role(), overwrite=True)

'Store suceess'

#### 1.1 role-lambda

In [7]:
listService = ["lambda.amazonaws.com"]
strRoleName = "DAT-LabmdaRole"
listPolicyArn = [
    "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
    "arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess"
]
strDescription="Role for Lambda to call SageMaker functions'"

In [8]:
lambdaRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)
pm.put_params(key="DAT-LAMBDA-ROLE-ARN", value=lambdaRoleArn, overwrite=True)
print (f"DAT-LAMBDA-ROLE-ARN: {lambdaRoleArn}")

== CREATE ROLE ==
  Role Name: [DAT-LabmdaRole] is already exist!!, so, this will be deleted and re-created.
  Service name: ['lambda.amazonaws.com'], 
  Role name: DAT-LabmdaRole, 
  Policys: ['arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole', 'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess', 'arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess'], 
  Role ARN: arn:aws:iam::419974056037:role/DAT-LabmdaRole
== COMPLETED ==
DAT-LAMBDA-ROLE-ARN: arn:aws:iam::419974056037:role/DAT-LabmdaRole


#### 1.2 role-codebuild

In [9]:
listService = ["codebuild.amazonaws.com"]
strRoleName = "DAT-CodeBuildRole"
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/CloudWatchFullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
]
strDescription="Role for CodeBuild"

In [10]:
buildRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)
pm.put_params(key="DAT-CODE-BUILD-ROLE-ARN", value=buildRoleArn, overwrite=True)
print (f"DAT-CODE-BUILD-ROLE-ARN: {buildRoleArn}")

== CREATE ROLE ==
  Role Name: [DAT-CodeBuildRole] is already exist!!, so, this will be deleted and re-created.
  Service name: ['codebuild.amazonaws.com'], 
  Role name: DAT-CodeBuildRole, 
  Policys: ['arn:aws:iam::aws:policy/AWSCodeCommitFullAccess', 'arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess', 'arn:aws:iam::aws:policy/AmazonS3FullAccess', 'arn:aws:iam::aws:policy/CloudWatchFullAccess', 'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess', 'arn:aws:iam::aws:policy/AWSLambda_FullAccess', 'arn:aws:iam::aws:policy/AmazonSSMFullAccess'], 
  Role ARN: arn:aws:iam::419974056037:role/DAT-CodeBuildRole
== COMPLETED ==
DAT-CODE-BUILD-ROLE-ARN: arn:aws:iam::419974056037:role/DAT-CodeBuildRole


#### 1.3 role-codepipeline

In [11]:
listService = ["codepipeline.amazonaws.com", "codebuild.amazonaws.com"]
strRoleName = "DAT-CodePipelineRole"
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodeDeployFullAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess"
]
strDescription="Role for CodePipeline"

In [12]:
pipelineRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)
pm.put_params(key="DAT-CODE-PIPELINE-ROLE-ARN", value=pipelineRoleArn, overwrite=True)
print (f"DAT-CODE-PIPELINE-ROLE-ARN: {pipelineRoleArn}")

== CREATE ROLE ==
  Role Name: [DAT-CodePipelineRole] is already exist!!, so, this will be deleted and re-created.
  Service name: ['codepipeline.amazonaws.com', 'codebuild.amazonaws.com'], 
  Role name: DAT-CodePipelineRole, 
  Policys: ['arn:aws:iam::aws:policy/AWSCodeCommitFullAccess', 'arn:aws:iam::aws:policy/AmazonS3FullAccess', 'arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess', 'arn:aws:iam::aws:policy/AWSCodeDeployFullAccess', 'arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess'], 
  Role ARN: arn:aws:iam::419974056037:role/DAT-CodePipelineRole
== COMPLETED ==
DAT-CODE-PIPELINE-ROLE-ARN: arn:aws:iam::419974056037:role/DAT-CodePipelineRole


In [16]:
!pip list

Package                            Version
---------------------------------- --------------------
aioboto3                           9.3.0
aiobotocore                        2.0.1
aiohttp                            3.8.1
aioitertools                       0.8.0
aiosignal                          1.2.0
alabaster                          0.7.12
anaconda-client                    1.8.0
anaconda-project                   0.10.2
anyio                              3.4.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        21.1.0
arrow                              1.2.1
asn1crypto                         1.4.0
astroid                            2.9.0
astropy                            5.0
async-generator                    1.10
async-timeout                      4.0.1
atomicwrites                       1.4.0
attrs                              21.2.0
Automat                            20.2.0
autopep8                           1.